In [1]:
import time
import matplotlib
from barMap import BarMap
matplotlib.use('TkAgg')  # Or 'Qt5Agg', 'Qt4Agg'
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from matplotlib.widgets import Button
from matplotlib.widgets import TextBox

# Create a figure and axis
fig, ax = plt.subplots()
fig.set_size_inches(10,6)

# Add padding around the graph using subplots_adjust
plt.subplots_adjust(left=0.25)

# Number of bars
n_bars = 2
waitingTime = 1

# X positions of the bars
x = [0,20]
nums = [56,23]

# Heights of the bars
#y = np.random.randint(1, 100, size=n_bars)

# Create the bar plot
bars = ax.bar(x, nums)
barPatches = bars.patches

# dictionary, value to bar rendered
barDictionary = {}
for height, patch in zip(nums, barPatches):
    patch.set_color('blue') #default color
    barDictionary[height] = patch

myBarMap = BarMap(barDictionary)

# Set up axis limits
ax.set_xlim(-1, 50)  # Extra room for shifting
ax.set_ylim(0, max(nums) + 2)

def updateTotal(expression):
    global bars,x,n_bars
    n_bars = int(expression)
    x = np.arange(n_bars)
    y = np.random.randint(1, 10, size=n_bars)
    ax.clear()
    bars = ax.bar(x, nums)
    # Set up axis limits
    ax.set_xlim(-1, n_bars)  # Extra room for shifting
    ax.set_ylim(0, max(nums) + 2)
    plt.show()


def start(event):
    global anim
    for i in range(0,len(nums)):
        if nums[i] > nums[i+1]:
            print("swapping!")
            myBarMap.AnimateSwap(nums[i], nums[i+1])
            t = nums[i]
            nums[i] = nums[i+1]
            nums[i+1] = t
            


    #myBarMap.MoveBarByYValue(23, 20)
    #myBarMap.MoveBarByYValue(56, 2)

text_box_ax = plt.axes([0.1, 0.5, 0.1, 0.05])
text_box = TextBox(text_box_ax, 'Total numbers:')
text_box.on_submit(updateTotal)

button_ax = plt.axes([0.01, 0.04, 0.08, 0.075])  # Position for the button
button = Button(button_ax, 'Start')
button.on_clicked(start)

def animate(i):
    myBarMap.Update()
    #if myBarMap.IsIdle():
    #    newX = np.random.randint(5, 20)
    #    myBarMap.MoveBarByYValue(23, newX)
    return bars

anim = FuncAnimation(fig, animate, frames=100, interval=5, blit=True)

# Display the animation
plt.show()
plt.close('all')





swapping!


Traceback (most recent call last):
  File "c:\Users\pferr\AppData\Local\Programs\Python\Python312\Lib\site-packages\matplotlib\cbook.py", line 298, in process
    func(*args, **kwargs)
  File "c:\Users\pferr\AppData\Local\Programs\Python\Python312\Lib\site-packages\matplotlib\widgets.py", line 244, in <lambda>
    return self._observers.connect('clicked', lambda event: func(event))
                                                            ^^^^^^^^^^^
  File "C:\Users\pferr\AppData\Local\Temp\ipykernel_9980\1527943687.py", line 63, in start
    myBarMap.AnimateSwap(nums[i], nums[i+1])
  File "d:\Work\Python\sorting\barMap.py", line 65, in AnimateSwap
    self.barDictionary.MoveBarByYValue(first, self.GetBarCenterXForValue(second))
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'dict' object has no attribute 'MoveBarByYValue'
